In [ ]:
pip install darts

     |████████████████████████████████| 921kB 6.5MB/s eta 0:00:01
     |████████████████████████████████| 307kB 20.1MB/s eta 0:00:01
     |███████████████                 | 375.1MB 119.5MB/s eta 0:00:04    |██████████▏                     | 254.2MB 143.5MB/s eta 0:00:04

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |███████████████████████████     | 671.1MB 420kB/s eta 0:05:0002�████████▋           | 514.5MB 114.4MB/s eta 0:00:03

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 797.1MB 15kB/s /s eta 0:00:01
     |████████████████████████████████| 317kB 82.8MB/s eta 0:00:01
     |████████████████████████████████| 13.4MB 111.3MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 229kB/s  eta 0:00:01
     |████████████████████████████████| 1.1MB 96.0MB/s eta 0:00:01
     |████████████████████████████████| 51kB 1.1MB/s  eta 0:00:01
     |████████████████████████████████| 542kB 96.7MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 65.0MB/s eta 0:00:01
     |████████████████████████████████| 10.8MB 66.3MB/s eta 0:00:01
     |████████████████████████████████| 8.3MB 85.7MB/s eta 0:00:01
     |████████████████████████████████| 174kB 79.0MB/s eta 0:00:01
     |████████████████████████████████| 819kB 97.3MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 89.2MB/s eta 0:00:01
     |████████████████████████████████| 4.5MB 69.0MB/s eta 0:00:01
     |████████████████████████████████| 102kB 1.7MB/s et

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 664.8MB 15kB/s /s eta 0:00:01
     |████████████████████████████████| 23.7MB 95.2MB/s eta 0:00:01
     |████████████████████████████████| 14.1MB 48.7MB/s eta 0:00:01
     |████████████████████████████████| 56.5MB 82.9MB/s eta 0:00:01
     |████████████████████████████████| 196.0MB 75kB/s s eta 0:00:010:01
     |████████████████████████████████| 209.4MB 36kB/s  eta 0:00:0101
     |████████████████████████████████| 829kB 82.4MB/s eta 0:00:01
     |████████████████████████████████| 410.6MB 32kB/s  eta 0:00:0101     |███████████▋                    | 149.6MB 125.3MB/s eta 0:00:03 207.5MB 139.6MB/s eta 0:00:02��██████████████████▋            | 252.2MB 111.3MB/s eta 0:00:02


In [ ]:
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [ ]:
my_session.connection.database

In [ ]:
my_session.connection.schema

In [ ]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [ ]:
sf_df = my_session.sql("select * from {}".format(table_name))
type(sf_df)
df=sf_df.to_pandas()
type(df)

In [ ]:
df.head(5)

In [ ]:
df.info()

## Generate Additional Features

In [ ]:
import pandas as pd

def generate_features(df):
    # Ensure the TRANS_DATE column is in datetime format
    df['TRANS_DATE'] = pd.to_datetime(df['TRANS_DATE'])
    
    # Extract temporal features
    df['year'] = df['TRANS_DATE'].dt.year
    df['month'] = df['TRANS_DATE'].dt.month
    df['day'] = df['TRANS_DATE'].dt.day
    df['dayofweek'] = df['TRANS_DATE'].dt.dayofweek
    df['quarter'] = df['TRANS_DATE'].dt.quarter
    df['is_month_start'] = df['TRANS_DATE'].dt.is_month_start
    df['is_month_end'] = df['TRANS_DATE'].dt.is_month_end
    
    # Create UNIT_PTR feature
    df['UNIT_PTR'] = df['SALES_PTR_VALUE'] / df['SALES_UNITS']
    
    return df

In [ ]:
df = generate_features(df)

In [ ]:
df['PRODUCT_CODE'].nunique()

In [ ]:
df = df.sort_values(by='TRANS_DATE')

In [ ]:
# Convert all column names to uppercase
df.columns = [col.upper() for col in df.columns]

In [ ]:
df.info()

In [ ]:
df_sorted = df.sort_values(by=['OUTLET_CODE', 'PRODUCT_CODE', 'TRANS_DATE'])

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# List of columns in the desired order
columns_order = ['OUTLET_CODE', 'PRODUCT_CODE', 'TRANS_DATE', 'UNIT_PTR'] + [col for col in df_sorted.columns if col not in ['OUTLET_CODE', 'PRODUCT_CODE']]

# Reorder the DataFrame columns
df_sorted = df_sorted[columns_order]

In [ ]:
df_sorted['FREQUENCY'] = df_sorted.groupby(['OUTLET_CODE', 'PRODUCT_CODE']).cumcount() + 1

In [ ]:
df_sorted.head()

Split the dataset into test and train

K means clustering of OUTLET_CODES

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def label_encode_dataframe(df, columns_to_encode):
    # Initialize the label encoder
    label_encoder = LabelEncoder()
    
    # Apply label encoding to each column
    for column in columns_to_encode:
        df[column + '_encoded'] = label_encoder.fit_transform(df[column])
    
    # Convert all column names to uppercase and replace spaces with underscores
    df.columns = df.columns.str.upper().str.replace(' ', '_')
    
    return df

In [ ]:
columns_to_encode = ['PRODUCT_CODE', 'CATEGORY', 'SUBCATEGORY', 'BRAND']
df_sorted = label_encode_dataframe(df_sorted, columns_to_encode)

In [ ]:
df_sorted.info()

In [ ]:
df_sorted = df_sorted.loc[:, ~df_sorted.columns.duplicated()]

In [ ]:
# Aggregation dictionary
aggregation_dict = {
    'SALES_UNITS': 'mean',
    'UNIT_PTR': 'mean',
    'FREQUENCY': 'count',
    'PRODUCT_CODE_ENCODED': 'mean'
}

# Aggregate data by OUTLET_CODE
aggregated_df = df_sorted.groupby('OUTLET_CODE').agg(aggregation_dict).reset_index()

In [ ]:
# Select features for clustering
features = ['SALES_UNITS', 'UNIT_PTR', 'FREQUENCY', 'PRODUCT_CODE_ENCODED']

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['SALES_UNITS', 'UNIT_PTR', 'FREQUENCY', 'PRODUCT_CODE_ENCODED']),
        # No need to preprocess category columns as they are already one-hot encoded
    ],
    remainder='passthrough'  # Keep the one-hot encoded columns as they are
)

In [ ]:
df_sorted_preprocessed = preprocessor.fit_transform(aggregated_df[features])

In [ ]:
# Calculate silhouette scores for different numbers of clusters
silhouette_scores = []
k_range = range(2, 11)  # Silhouette score is not defined for k=1
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(df_sorted_preprocessed)
    silhouette_avg = silhouette_score(df_sorted_preprocessed, cluster_labels)
    silhouette_scores.append(silhouette_avg)

# Plot the silhouette scores
plt.figure(figsize=(10, 6))
plt.plot(k_range, silhouette_scores, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score For Optimal k')
plt.show()

In [ ]:
# Apply K-Means clustering with the chosen number of clusters 
optimal_k = k_range[silhouette_scores.index(max(silhouette_scores))]  # Choose the k with the highest silhouette score
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
aggregated_df['CLUSTER'] = kmeans.fit_predict(df_sorted_preprocessed)

# Display the first few rows to verify
print(aggregated_df.tail())

In [ ]:
aggregated_df.to_csv('outlet_to_cluster_mapping.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Example visualization Clusters of SALES_UNITS VS SALES_VALUE
sns.scatterplot(x='SALES_UNITS', y='FREQUENCY', hue='CLUSTER', data=aggregated_df)
plt.title('Clusters of OUTLET_CODE')
plt.show()

In [ ]:
df_sorted.head()

In [ ]:
df_sorted=df_sorted.merge(aggregated_df[['OUTLET_CODE', 'CLUSTER']], on='OUTLET_CODE', how='left')

In [ ]:
df_sorted.to_csv('HistoricalSales_with_cluster.csv', index=False)

In [ ]:
unique_product_codes = df['PRODUCT_CODE'].unique()
df_unique_product_codes = pd.DataFrame(unique_product_codes, columns=['PRODUCT_CODE'])

unique_outlet_codes = df['OUTLET_CODE'].unique()
df_unique_outlet_codes = pd.DataFrame(unique_outlet_codes, columns=['OUTLET_CODE'])

df_all_combinations = df_unique_product_codes.merge(df_unique_outlet_codes, how='cross')

current_selling = df_sorted[['PRODUCT_CODE', 'OUTLET_CODE']].drop_duplicates()

In [ ]:
# Assuming df_all_combinations and unique_combinations are your DataFrames
# Merge the DataFrames with an indicator
merged_df = df_combinations.merge(current_selling, on=['PRODUCT_CODE', 'OUTLET_CODE'], how='left', indicator=True)

# Filter the rows that are present in df_combinations but not in unique_combinations
diff_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])


In [ ]:
diff_df.info()

In [ ]:
diff_df=diff_df.merge(aggregated_df[['OUTLET_CODE', 'CLUSTER']], on='OUTLET_CODE', how='left')

In [ ]:
diff_df.to_csv('Not_selling.csv', index=False)